# PostGIS on Python 3 and DBSCAN implementation
In this project, we use the dataset "animale" which contains a trajectory of 1189 points taken from a roe deer. 
The positions are expressed in projected coordinates (EPSG 3857).
We have to implement DBSCAN algorithm in order to find the most densely populated areas from the animal.
The result must be a list of the original points associated to a label. The labels denote the cluster each point belongs to. 
A label equals to 0 denotes a noisepoint.
<br>

Developers: [Luca Arrotta](https://github.com/lucaArrotta), 
[Cheick Tidiane Ba](), 
[Giovanni Laganà](https://bitbucket.org/giolaga/).

### PostGIS on Python 3

Create the connection with the database.

In [1]:
def create_connection():
    import psycopg2
    try:
        return psycopg2.connect("dbname='progetto' user='postgres' host='localhost' password='postgres' port=5433")
    except psycopg2.Error as e:
        print("Unable to connect to the database")
        print(e)

conn = create_connection()

<br>
Test1: get the name of each db.

In [3]:
cur = conn.cursor()
cur.execute("""
    SELECT datname 
    FROM pg_database""")

rows = cur.fetchall()
print ("Databases:")

for row in rows:
    print("\t", row[0])

Databases:
	 postgres
	 esercitazioni
	 template1
	 template0
	 lab9
	 lab10
	 lab13
	 progetto


<br> 
Test2: get 10 rows from table Animale(gid, animal, id, time, x, y, geom).

In [5]:
cur = conn.cursor()
cur.execute("""
    SELECT * 
    FROM animale 
    LIMIT 10""")
rows = cur.fetchall()
print ("Animal table:")

for row in rows:
    print(row)

Animal table:
(1, '767', 1, '2005-11-27 16:01:53', Decimal('1231569.438434269977733'), Decimal('5781498.869216189719737'), '0101000020110F0000713A3D70D1CA3241F43CA1B7FE0D5641')
(2, '767', 2, '2005-11-28 00:03:00', Decimal('1231541.552901820046827'), Decimal('5781371.871003749780357'), '0101000020110F000057F98A8DB5CA32418486BEF7DE0D5641')
(3, '767', 3, '2005-11-28 04:01:52', Decimal('1231879.106993759982288'), Decimal('5781535.539407029747963'), '0101000020110F00005FF1631B07CC32410BA585E2070E5641')
(4, '767', 4, '2005-11-28 08:01:24', Decimal('1231599.283189750043675'), Decimal('5781672.877671480178833'), '0101000020110F00009B1F7F48EFCA3241FBC42B382A0E5641')
(5, '767', 5, '2005-11-28 12:01:56', Decimal('1231634.192982059903443'), Decimal('5781679.464961109682918'), '0101000020110F0000C245673112CB324140ECC1DD2B0E5641')
(6, '767', 6, '2005-11-28 16:01:24', Decimal('1231462.872285729972646'), Decimal('5781463.401208249852061'), '0101000020110F0000231E4EDF66CA32415B65ADD9F50D5641')
(7, '767

<br>
Test3: get distances between point with gid=1 and all other points (considering only 10 rows).

In [9]:
diz_points = {
    "gid_point1": 0,
    "gid_point2": 1,
    "distance": 2,
    "geom1": 3,
    "geom2": 4
}

cur = conn.cursor()
cur.execute("""
    SELECT a1.gid, a2.gid, ST_distance(a1.geom,a2.geom), a1.geom, a2.geom
    FROM animale a1, animale a2
    WHERE a1.gid = 1 
    LIMIT 10""")

rows = cur.fetchall()

index = diz_points["distance"]
print("Distances from point with gid=1:")

for row in rows:
    print("\tgid {}  ->  {}".format(row[diz_points["gid_point2"]], row[diz_points["distance"]]))

Distances from point with gid=1:
	gid 1  ->  0.0
	gid 2  ->  130.023647399785
	gid 3  ->  311.832197873674
	gid 4  ->  176.549290396964
	gid 5  ->  191.854044895645
	gid 6  ->  112.313505874285
	gid 7  ->  154.208097192093
	gid 8  ->  64.7523210546744
	gid 9  ->  254.779609133287
	gid 10  ->  166.266620603794


<br>
Test4: create a table.

In [10]:
cur = conn.cursor()
sql = """CREATE TABLE test_table AS (
            SELECT *
            FROM animale
        )"""
cur.execute(sql)
conn.commit() # makes sure the change is shown in the database

TODO
TODO
TODO
Test5: add column to table.

In [11]:
cur.execute('ALTER TABLE test_table ADD "label" INTEGER')
conn.commit()
# QUI MANCA DA AGGIUNGERE UN VALORE ALLA LABEL

In [21]:
cur = conn.cursor()
cur.execute("""
    SELECT * 
    FROM test_table 
    LIMIT 10""")
rows = cur.fetchall()
print ("Test table:")

for row in rows:
    print(row)

Test table:
(1, '767', 1, '2005-11-27 16:01:53', Decimal('1231569.438434269977733'), Decimal('5781498.869216189719737'), '0101000020110F0000713A3D70D1CA3241F43CA1B7FE0D5641', None)
(2, '767', 2, '2005-11-28 00:03:00', Decimal('1231541.552901820046827'), Decimal('5781371.871003749780357'), '0101000020110F000057F98A8DB5CA32418486BEF7DE0D5641', None)
(3, '767', 3, '2005-11-28 04:01:52', Decimal('1231879.106993759982288'), Decimal('5781535.539407029747963'), '0101000020110F00005FF1631B07CC32410BA585E2070E5641', None)
(4, '767', 4, '2005-11-28 08:01:24', Decimal('1231599.283189750043675'), Decimal('5781672.877671480178833'), '0101000020110F00009B1F7F48EFCA3241FBC42B382A0E5641', None)
(5, '767', 5, '2005-11-28 12:01:56', Decimal('1231634.192982059903443'), Decimal('5781679.464961109682918'), '0101000020110F0000C245673112CB324140ECC1DD2B0E5641', None)
(6, '767', 6, '2005-11-28 16:01:24', Decimal('1231462.872285729972646'), Decimal('5781463.401208249852061'), '0101000020110F0000231E4EDF66CA324

Close the connection.

In [12]:
conn.close()
cur.close()

***

### DBSCAN implementation

1) Scriviamo una funzione che restituisce una matrice con le distanze tra tutte le coppie di punti possibili.

In [13]:
def get_distance_matrix(conn):
    cur = conn.cursor()
    cur.execute("""
        SELECT a1.gid, a2.gid, ST_distance(a1.geom,a2.geom)
        FROM animale a1, animale a2""")
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    df.columns = ["gid_point1", "gid_point2", "distance"]
    return df

import pandas as pd
conn = create_connection()
display(get_distance_matrix(conn))

,gid_point1,gid_point2,distance
0,1,1,0.000000
1,1,2,130.023647
2,1,3,311.832198
3,1,4,176.549290
4,1,5,191.854045
5,1,6,112.313506
6,1,7,154.208097
7,1,8,64.752321
8,1,9,254.779609
9,1,10,166.266621


<br>

2) funzione che restituisce un dizionario in cui all'id di ogni punto è associato il suo stato (core point, border point, noise point)

In [20]:
def get_len(conn):
    ''' restituisce il numero di punti del dataset '''
    cur = conn.cursor()
    query = "SELECT count(*) as len\
            FROM animale"
    cur.execute(query)
    length = cur.fetchall()[0][0]
    return length


def get_core_points(eps, minPoints, conn):
    '''
    restituisce una lista con gli id di ogni core point trovato
    '''
    cur = conn.cursor()
    query = "SELECT a1.gid\
            FROM animale a1, animale a2\
            WHERE ST_distance(a1.geom,a2.geom)<={}\
            GROUP BY a1.gid\
            HAVING count(*)>={}+1".format(eps, minPoints)
    cur.execute(query)
    rows = cur.fetchall()
    core_points_list = list()
    for row in rows:
        core_points_list.append(row[0])
    return core_points_list


def get_neighbors(eps, point):
    '''
    '''
    cur = conn.cursor()
    query = "SELECT a2.gid\
            FROM animale a1, animale a2\
            WHERE ST_distance(a1.geom,a2.geom) <= {} and a1.gid = {}".format(eps, point)
    cur.execute(query)
    rows = cur.fetchall()
    neighbors_list = list()
    for row in rows:
        neighbors_list.append(row[0])
    return neighbors_list
    


def expand_cluster(point, cluster, eps, minPoints, labels):
    '''
    '''
    neighbors_list1 = get_neighbors(eps, point)
    for i, el in enumerate(neighbors_list1):
        if el == 0 or el == -1:  # allora il punto non è ancora stato visitato o è rumore per ora
            labels[i] = cluster
            neighbors_list2 = get_neighbors(eps, el)
            if len(neighbors_list2) >= minPoints:
                print()
    return
        
            
def dbscan(eps, minPoints, conn):
    '''
    '''
    len_dataset = get_len(conn)
    labels = [0 for i in range(len_dataset)]  # lista con 0 associato ad ogni punto del dataset
    core_points_list = get_core_points(eps, minPoints, conn)
    
    cluster = 0
    for i, el in enumerate(labels):  # nota: i -> corrisponde all'id del punto
        if el == 0:  # allora il punto non è ancora stato visitato
            if i in core_points_list:  # allora il punto è un core point
                cluster += 1
                labels[i] = cluster
                expand_cluster(i, cluster, eps, minPoints, labels)
            else:  # allora il punto è noise point o border point (se è un border point, allora verrà raggiunto durante l'espansion di uno dei cluster)
                labels[i] = -1
    return
    
    
dbscan(300, 100, conn)
#d = get_core_points(300, 100, conn)
#print(d)